In [1]:
import os
import shutil
import sys
import time
from PIL import Image
from datetime import date, datetime, timedelta


sys.path.insert(0, '..')
from utils import mongodb
from vazios_cheios import cursor_vazio_nvazio, extract_to


In [2]:
if os.path.exists('vazios'):
    shutil.rmtree('vazios')

## Generate vazios database

In [3]:
start = datetime(2017, 7, 1)
end = datetime.today()
limit = 500
for label in (True, False):
    s0 = time.time()
    cursor = cursor_vazio_nvazio(mongodb, start, end, limit, label)
    count = extract_to(cursor)
    s1 = time.time()
    print('{:0.2f} segundos para processar {:d} registros'.format((s1 - s0), count))


(600, 365)  - abortando...
(600, 365)  - abortando...
(600, 357)  - abortando...
(25, 400)  - abortando...
(14, 400)  - abortando...
(600, 338)  - abortando...
(600, 350)  - abortando...
(600, 337)  - abortando...
(600, 352)  - abortando...
(600, 338)  - abortando...
(600, 352)  - abortando...
(600, 335)  - abortando...
(600, 350)  - abortando...
(512, 400)  - abortando...
(538, 400)  - abortando...
(600, 346)  - abortando...
(600, 349)  - abortando...
(600, 345)  - abortando...
(600, 375)  - abortando...
(600, 341)  - abortando...
(600, 376)  - abortando...
(600, 390)  - abortando...
(600, 387)  - abortando...
(600, 381)  - abortando...
(600, 338)  - abortando...
(600, 348)  - abortando...
(600, 338)  - abortando...
(600, 367)  - abortando...
(600, 334)  - abortando...
(600, 337)  - abortando...
(600, 342)  - abortando...
499 arquivos exportados...
8.79 segundos para processar 499 registros
(600, 353)  - abortando...
(600, 357)  - abortando...
(600, 343)  - abortando...
(600, 358)  - 

## Example: Training a very basic convolutional network

In [4]:
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau


In [5]:
model = models.Sequential([
  layers.Conv2D(16, (3, 3), activation='relu',
                padding='same',
                input_shape=(144, 144, 3)),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Dropout(0.25),
  layers.Conv2D(128, (3, 3), activation='relu'),
  layers.Conv2D(128, (1, 1), activation='relu'),
  layers.MaxPooling2D(pool_size=(2, 2)),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(256, (3, 3), activation='relu'),
  layers.Conv2D(256, (1, 1), activation='relu'),
  layers.Flatten(),
  layers.Dense(256, activation='relu'),
  layers.Dropout(0.2),
  layers.Dense(1, activation='sigmoid')
 
])

model.compile(optimizer=optimizers.RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 144, 144, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 72, 72, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 72, 72, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 64)        1

In [9]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest',
    horizontal_flip=True,
    validation_split=0.2
)
train_generator = train_datagen.flow_from_directory(
    'vazios',
    target_size=(144, 144),
    batch_size=100,
    class_mode='binary',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    'vazios',
    target_size=(144, 144),
    batch_size=50,
    class_mode='binary',
    subset='validation'
)


Found 761 images belonging to 2 classes.
Found 189 images belonging to 2 classes.


In [10]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5,
                              verbose=1, min_delta=1e-2, mode='min')

model.fit_generator(train_generator, epochs=50,
                    callbacks=[reduce_lr],
                   validation_data = validation_generator,
                   validation_steps = validation_generator.samples // 100)

Epoch 1/50
8/8 [==============================] - 14s 2s/step - loss: 0.2218 - acc: 0.9396 - val_loss: 0.6177 - val_acc: 0.5400
Epoch 2/50
8/8 [==============================] - 11s 1s/step - loss: 0.3893 - acc: 0.8121 - val_loss: 0.5396 - val_acc: 0.5400
Epoch 3/50
8/8 [==============================] - 13s 2s/step - loss: 0.2903 - acc: 0.8804 - val_loss: 0.4787 - val_acc: 0.5400
Epoch 4/50
8/8 [==============================] - 12s 1s/step - loss: 0.2307 - acc: 0.9225 - val_loss: 0.3939 - val_acc: 0.8200
Epoch 5/50
8/8 [==============================] - 12s 1s/step - loss: 0.2920 - acc: 0.8922 - val_loss: 0.4050 - val_acc: 0.9800
Epoch 6/50
8/8 [==============================] - 12s 1s/step - loss: 0.1969 - acc: 0.9501 - val_loss: 0.4409 - val_acc: 0.7400
Epoch 7/50
8/8 [==============================] - 12s 1s/step - loss: 0.3100 - acc: 0.8962 - val_loss: 0.4696 - val_acc: 0.5800
Epoch 8/50
8/8 [==============================] - 11s 1s/step - loss: 0.1685 - acc: 0.9606 - val_loss: 0